In [ ]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import math
from astropy import time, coordinates as coord, units as u
import requests
import matplotlib.dates as mdates
from pandas.plotting import table

Path to PixInsight photometry result files

In [ ]:
target="TW Cas"
path="D:\\Astronomie\\SharpCap Captures\\20241104\\TW Cas\\work\\photometry"
filelist=os.listdir(path)

Get AAVSO data from target variable

In [ ]:
response = requests.get(f"http://www.aavso.org/vsx/index.php?view=api.object&ident={target}&format=json")   
vsx_data=response.json()['VSXObject']
vsx_data

In [ ]:
constellation=vsx_data['Constellation']
star=vsx_data['Name']
period= vsx_data['Period']
position = coord.SkyCoord(vsx_data['RA2000'],vsx_data['Declination2000'],
                        unit=(u.deg, u.deg), frame='icrs')
observatory = coord.EarthLocation(lat=51.43, lon=6.82, height=40)

Get Photometry field from AAVSO

In [ ]:
response = requests.get(f"https://www.aavso.org/vsp/api/chart/?star={star}&fov=60&maglimit=15&format=json")
chart_data=response.json()
chart_data

In [ ]:
auid_list = []
for photometry_data in chart_data['photometry']:
    auid = photometry_data['auid']
    ra=photometry_data['ra']
    dec=photometry_data['dec']
    for band_data in photometry_data['bands']:
        if band_data['band'] == 'V':
            mag = band_data['mag']
            auid_list.append((auid, ra, dec, mag))

In [ ]:
auid_list

Match with PixInsight Photometry Table

In [ ]:
file=filelist[0]
df=pd.read_csv(path+"\\"+file,sep=";",skiprows=5)
df.columns = df.columns.str.replace(' ', '')
names= df[['NAME']].to_numpy()
catalogue= df[['CATRA', 'CATDEC']].to_numpy()
flags= df[['FLAG']].to_numpy() 
catalogue=coord.SkyCoord(catalogue[:,0],catalogue[:,1],unit=(u.deg, u.deg), frame='icrs')

In [ ]:
idx,d2d,d3d=position.match_to_catalog_sky(catalogue)
varstar=names[idx][0].replace(" ", "")
varstar

In [ ]:
star_data=np.array([(tup[1], tup[2]) for tup in auid_list])
star_coord=coord.SkyCoord(star_data[:,0],star_data[:,1],unit=(u.hourangle, u.deg), frame='icrs')

In [ ]:
idx,d2d,d3d=star_coord.match_to_catalog_sky(catalogue)

In [ ]:
d2d

In [ ]:
df.iloc[idx]

In [ ]:
photometry_stars=[]
photometry_mags=[]
for i in range(len(auid_list)):
    if d2d[i].arcsec < 10.0:
        photometry_stars.append(names[idx[i]][0].replace(" ", ""))
        photometry_mags.append(auid_list[i][3])

In [ ]:
photometry_stars, photometry_mags

In [ ]:
photometry_stars.append('3UCAC312-021454')
photometry_mags.append(9.957)

In [ ]:

check=photometry_stars[2]
compstar=photometry_stars[0:2]
compstar_mag=photometry_mags[0:2]

In [ ]:
compstar

In [ ]:
file=filelist[0]
df=pd.read_csv(path+"\\"+file,sep=";",skiprows=5)
df

In [ ]:
t0=df.iloc[0,0]
t0_iso=time.Time(t0,format="jd")

In [ ]:
l=[]
for s in df.iloc[:,1]: l=l+[s.strip()]
df.iloc[:,1]=l

In [ ]:
def snrlst(target,infiles):
    target_snr=[]
    for file in infiles:
        df=pd.read_csv(path+"\\"+file,sep=";",skiprows=5)
         #find target star
        l=[]
        for s in df.iloc[:,1]: l=l+[s.strip()]
        df.iloc[:,1]=l
        itarget=(df.iloc[:,1]==target)
        target_snr=target_snr+[df.loc[itarget]]
    target_snr=pd.concat(target_snr,ignore_index=True)
    ind=[]
    col=target_snr.columns.to_list()
    for s in col:
        ind = ind + [s.startswith("SNR")]
    target_snr=target_snr.loc[:,ind]
    return target_snr.mean()

In [ ]:
snr=snrlst(varstar,filelist)
snrmax=snr.idxmax()
labelflux=snrmax.replace("SNR","FLUX")
labelflux

In [ ]:
def fluxcalc2(target,infiles,label_max):
    target_flux=pd.DataFrame()
    for file in infiles:
        #read file
        df=pd.read_csv(path+"\\"+file,sep=";",skiprows=5)
        #find target star
        l=[]
        for s in df.iloc[:,1]: l=l+[s.strip()]
        df.iloc[:,1]=l
        itarget=(df.iloc[:,1]==target)
        #read flux
        flux=df.loc[itarget][['DATE_OBS     ',label_max]]
        flux=flux.rename(columns={'DATE_OBS     ':"DATE_OBS",label_max:"FLUX"})
        target_flux=pd.concat([target_flux,flux])
    target_flux=target_flux.reset_index(drop=True)
    return target_flux

In [ ]:
minind=0
maxind=len(filelist)

In [ ]:
flux_var=fluxcalc2(varstar,filelist,labelflux)
flux_check=fluxcalc2(check,filelist,labelflux)

In [ ]:
flux_comp=pd.DataFrame()
flux_comp["DATE_OBS"]=flux_var["DATE_OBS"]
flux_comp=flux_comp.reset_index(drop=True)
for star in compstar:
    flux=fluxcalc2(star,filelist,labelflux)
    #flux = flux.reset_index(drop=True)
    flux_comp[star]=flux['FLUX']
print(flux_comp)   

In [ ]:
t=flux_var.to_numpy()[minind:maxind,0]
mag_var=flux_var.to_numpy()[minind:maxind,1]
mag_check=flux_check.to_numpy()[minind:maxind,1]
mag_comp=flux_comp.to_numpy()[minind:maxind,1:]

In [ ]:
nstars = mag_comp.shape[1]+2
nstars

In [ ]:
flux_var.to_csv("flux_var.csv",index=False)

In [ ]:
fig, ax = plt.subplots(nstars,1,figsize=(5.8,8.3), layout='constrained')
fig.suptitle("raw data " + " " + target + " " + t0_iso.iso )
fig.supylabel("Flux [ADU]")
ax[0].plot(t,mag_var,"bo",ms=2)
ax[0].set_title(varstar)
ax[0].tick_params(axis="x",labelbottom=False)
ax[0].grid(visible="true")
ax[1].plot(t,mag_check,"bo",ms=2)
ax[1].set_title(check)
ax[1].grid(visible="true")
ax[1].tick_params(axis="x",labelbottom=False)
for i in range(2,nstars):
    ax[i].plot(t,mag_comp[:,i-2],"bo",ms=2)
    ax[i].set_title(compstar[i-2])
    ax[i].grid(visible="true")
    ax[i].tick_params(axis="x",labelbottom=False)
ax[nstars-1].set_xlabel("JD")
ax[nstars-1].tick_params(axis="x",labelbottom=True)

fig.savefig(constellation+"_"+target.replace(constellation, "")+"_"+'%-8.4f'%(t0_iso.to_value("jd")-2400000)+"_"+"RAW_DATA"+".pdf", format="pdf")

In [ ]:
flux_var

In [ ]:
flux_var["FLUX"]=flux_var["FLUX"].rolling(window=5,min_periods=1).mean()

In [ ]:
flux_comp[['3UCAC312-021632', '3UCAC312-021635']] = flux_comp[['3UCAC312-021632', '3UCAC312-021635']].rolling(window=5, min_periods=1).mean()
print(flux_comp)

In [ ]:
# Create a new DataFrame with the same column names as flux_comp
mag_var = pd.DataFrame({'DATE_OBS': flux_comp['DATE_OBS']})

# Calculate the new columns
for star, mag0 in zip(compstar, compstar_mag):
    mag_var[star] = -2.5 * np.log10(flux_var['FLUX'] / flux_comp[star]) + mag0

print(mag_var)

In [ ]:
# Identify the columns to be averaged (all columns except 'date_obs')
columns_to_average = mag_var.columns.difference(['DATE_OBS'])

# Calculate the row-wise mean for these columns
mag_var['average'] = mag_var[columns_to_average].mean(axis=1)

mag_var

In [ ]:
mag_var.to_csv(constellation+"_"+target.replace(constellation, "")+"_"+'%-8.4f'%(t0_iso.to_value("jd")-2400000)+"_"+"MAG_DATA"+".csv", sep=";", index=False)

In [ ]:
#mag = mag_var['average'].to_numpy()
mag = mag_var['3UCAC312-021635'].to_numpy()
mag.shape

In [ ]:
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(t,mag,"bo",ms=2)  # Plot some data on the axes.
#ax.set_ylim(12.7,12.8)
ax.invert_yaxis()
ax.grid(visible=True)

In [ ]:
# Create a new DataFrame with the same column names as flux_comp
mag_check = pd.DataFrame({'DATE_OBS': flux_comp['DATE_OBS']})

# Calculate the new columns
for star, mag0 in zip(compstar, compstar_mag):
    mag_check[star] = -2.5 * np.log10(flux_check['FLUX'] / flux_comp[star]) + mag0

print(mag_check)

In [ ]:
# Identify the columns to be averaged (all columns except 'date_obs')
columns_to_average = mag_check.columns.difference(['DATE_OBS'])

# Calculate the row-wise mean for these columns
mag_check['average'] = mag_check[columns_to_average].mean(axis=1)

mag_check

In [ ]:
mag_c = mag_check['average'].to_numpy()
mag.shape

In [ ]:
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(t,mag_c,"bo",ms=2)  # Plot some data on the axes.
ax.plot(t,mag,"bo",ms=2) 
ax.grid(visible=True)
#ax.set_ylim(12.4,12.5)
ax.invert_yaxis()

In [ ]:
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.plot(t,mag_c,"bo",ms=2)  # Plot some data on the axes.
#ax.plot(t,mag,"bo",ms=2) 
ax.grid(visible=True)
#ax.set_ylim(12.4,12.5)
ax.invert_yaxis()

In [ ]:
err_c=np.std(mag_c)
err_c

In [ ]:
lc=np.empty((mag.shape[0],3))
lc[:,0]=t
lc[:,1]=mag
lc[:,2]=err_c

In [ ]:
t0=int(lc[0,0])
t=lc[:,0]-t0
mag=lc[:,1]
merr=lc[:,2]

In [ ]:
t0_ap=time.Time(t0,format="jd")
t0_ap.to_value("iso",subfmt="date")

In [ ]:
p=float(period)
w=2*math.pi/p
k=3
C=np.stack([np.sin(i*w*t) for i in range(1,k+1)])
B=np.stack([np.cos(i*w*t) for i in range(k+1)])
A=np.concatenate((B,C)).T
coeff = np.linalg.lstsq(A,mag,rcond=None)[0]

In [ ]:
def lc_fourier(t,w,coeff):
    n=coeff.shape[0]
    mag=0
    k=int((n-1)/2)
    for i in range(0,k+1):
        mag=mag+coeff[i]*np.cos(i*w*t)
    for i in range(k+1,n):
        mag=mag+coeff[i]*np.sin((i-k)*w*t)
    return mag

In [ ]:
t_step=np.linspace(min(t),max(t),1000)

In [ ]:
x=lc_fourier(t_step,w,coeff)

In [ ]:
ind=sp.signal.argrelmin(x)
ind1=sp.signal.argrelmax(x)

In [ ]:
ind1

In [ ]:
t0_ap=time.Time(t0+t_step[ind],format="jd")
t1_ap=time.Time(t0+t_step[ind1],format="jd")
t1_ap.to_value("iso")

In [ ]:
t

In [ ]:
#import kvw
#result = kvw.kvw(t[50:220],10-mag[50:220], init_minflux=1,nfold=3,debug=3)
#result

In [ ]:
#t0_ap = time.Time(t0+t_step[ind],format="jd",
#                  scale='utc', location=observatory)  
t1_ap=time.Time(t0+t_step[ind1],format="jd",
                  scale="utc",location=observatory)
#ltt0_helio = t0_ap.light_travel_time(position,"heliocentric") 
ltt1_helio = t1_ap.light_travel_time(position,"heliocentric") 
ltt1_helio.to_value("jd") 

In [ ]:
time.Time(t0+t_step[ind1],format="jd",
                  scale='utc', location=observatory)  

In [ ]:
t_am=time.Time(t0+t_step,format="jd",scale="utc",location=observatory)
frame=coord.AltAz(obstime=t_am,location=observatory)
airmass=position.transform_to(frame).secz
[airmass[0],airmass[999],np.min(airmass)]

In [ ]:
description=[#["Maximum (UTC, geocentric)",t0_ap[0].to_value("iso")],
             #["Maximum (HJD, time base UTC)",'%-8.4f'%(t0_ap[0].to_value("jd")+ltt0_helio[0].to_value("jd"))+"+/-0.002"],
             #["Maximum (UTC, geocentric)",t0_ap[1].to_value("iso")],
             #["Maximum (HJD, time base UTC)",'%-8.4f'%(t0_ap[1].to_value("jd")+ltt0_helio[1].to_value("jd"))+"+/-0.002"],
             ["Minimum (UTC, geocentric)",t1_ap[0].to_value("iso")],
             ["Minimim (HJD, time base UTC)",'%-8.4f'%(t1_ap[0].to_value("jd")+ltt1_helio[0].to_value("jd"))+"+/-0.002"],
             #["Minimum (UTC, geocentric)",t1_ap[1].to_value("iso")],
             #["Minimim (HJD, time base UTC)",'%-8.4f'%(t1_ap[1].to_value("jd")+ltt1_helio[1].to_value("jd"))+"+/-0.002"],
             ["Observer","Deeskow, DES"],
             ["Instrumnent","CFF140 f/6.6, ASI1600mm with V Filter"],
             ["Comparison Star",compstar[0]],
#             ["",compstar[1]],
#             ["",compstar[2]],
             ["Photometry","PixInsight Photometry Skript"],
             ["Evaluation","Fourier-Fit"],
             ["Airmass","1.072 ... 1.028 ... 1.174"],
             ["Number of Measurements",str(lc.shape[0])]
            ]
title=target + " " + "             "+t1_ap[0].to_value("iso",subfmt="date")

In [ ]:
tiso=time.Time(t0+t,format="jd")
t_step_iso=time.Time(t0+t_step,format="jd")
jd0=t_step_iso[0].jd

In [ ]:
mpl0=mpl.dates.date2num(t_step_iso[0].datetime)

In [ ]:
def jd2mpl(tin):
    tout=tin+mpl0-jd0-ltt1_helio[0].to_value("jd")+int(jd0)
    return tout

In [ ]:
def mpl2jd(tin):
    tout=tin-mpl0+jd0+ltt1_helio[0].to_value("jd")-int(jd0) 
    return tout

In [ ]:
fig, (ax,ax1) = plt.subplots(2,1,height_ratios=[1,1],figsize=(5.8,8.3))
fig.suptitle(title,x=0.5,y=0.97,ha="center")
ax.clear()
ax.errorbar(tiso.datetime,mag,yerr=merr,elinewidth=0.5,marker="o",ms=2,linestyle="none")
ax.plot(t_step_iso.datetime,lc_fourier(t_step,w,coeff),"b--",linewidth=1)  
#for i in ind[0]:
#   ax.axvline(t_step_iso[i].datetime,0,1,color='dimgrey',ls="--")
for i in ind1[0]:
    ax.axvline(t_step_iso[i].datetime,0,1,color='dimgrey',ls="--")
ax.invert_yaxis()
ax.minorticks_on()
ax.grid(visible=True,which="both")
ax.set_xlabel("UTC")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
#ax.set_xlabel("JD "+'%-8.0f'%(int(t0_ap.to_value("jd"))))
ax.set_ylabel("Magnitude")

secax = ax.secondary_xaxis('top', functions=(mpl2jd,jd2mpl))
secax.set_xlabel('HJD'+" "+str(int(jd0))+" +")
#pticks=ax.get_xticks()
#secax.set_xticks(mpl2jd(pticks))
secax.minorticks_on()

ax1.tick_params(
    axis="both",
    which="both",
    bottom=False,
    top=False,
    left=False,
    right=False,
    labelbottom=False,
    labeltop=False,
    labelleft=False,
    labelright=False)
ax1.set_axis_off()
ax1.table(description,loc="upper center",cellLoc="left",edges="")

fig.savefig(constellation+"_"+target.replace(constellation, "")+"_"+'%-8.4f'%(t1_ap[0].jd-2400000+ltt1_helio[0].to_value("jd"))+"_"+"DES"+".pdf", format="pdf")
    

In [ ]:
fig, ax = plt.subplots(1,1,height_ratios=[1],figsize=(5.8,5.8))
fig.suptitle(title,x=0.5,y=0.97,ha="center")
ax.clear()
ax.errorbar(tiso.datetime,mag,yerr=merr,elinewidth=0.5,marker="o",ms=2,linestyle="none")
ax.plot(t_step_iso.datetime,lc_fourier(t_step,w,coeff),"b--",linewidth=1)  
for i in ind[0]:
    ax.axvline(t_step_iso[i].datetime,0,1,color='dimgrey',ls="--")
for i in ind1[0]:
    ax.axvline(t_step_iso[i].datetime,0,1,color='dimgrey',ls="--")
ax.invert_yaxis()
ax.minorticks_on()
ax.grid(visible=True,which="both")
ax.set_xlabel("UTC")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
#ax.set_xlabel("JD "+'%-8.0f'%(int(t0_ap.to_value("jd"))))
ax.set_ylabel("Magnitude")

secax = ax.secondary_xaxis('top', functions=(mpl2jd,jd2mpl))
secax.set_xlabel('HJD'+" "+str(int(jd0))+" +")
#pticks=ax.get_xticks()
#secax.set_xticks(mpl2jd(pticks))
secax.minorticks_on()


fig.savefig(constellation+"_"+target.replace(constellation, "")+"_"+'%-8.4f'%(t0_ap[0].jd-2400000+ltt0_helio[0].to_value("jd"))+"_"+"DES"+".png", format="png")
    

In [ ]:
maxlist=[]
for i in range(t0_ap.shape[0]): 
    maximum=[constellation,
         target.replace(constellation, ""),
         "max",
         '%-8.4f'%(t0_ap[i].to_value("jd")),
         '%-8.4f'%(t0_ap[i].to_value("jd")+ltt0_helio[i].to_value("jd")),
         '0.002',
         "na",
         "na",
         '%-6.3f'%(lc_fourier(t0_ap[i].jd-t0,w,coeff)),
         "C",
         "ASI1600MM",
         "V",
         str(lc.shape[0]),
         "DES",
         "na"
        ]
    maxout=""
    for entry in maximum:
        maxout=maxout+entry+"|"
    maxlist=maxlist+[maxout]   
        
maxlist

In [ ]:
minlist=[]
for i in range(t1_ap.shape[0]):
    minimum=[constellation,
         target.replace(constellation, ""),
         "min",
         '%-8.4f'%(t1_ap[i].to_value("jd")),
         '%-8.4f'%(t1_ap[i].to_value("jd")+ltt1_helio[i].to_value("jd")),
         '0.002',
         "na",
         "na",
         '%-6.3f'%(lc_fourier(t1_ap[i].jd-t0,w,coeff)),
         "C",
         "ASI1600MM",
         "V",
         str(lc.shape[0]),
         "DES",
         "na"
        ]
    minout=""
    for entry in minimum:
        minout=minout+entry+"|"
    minlist=minlist+[minout]  
minlist

In [ ]:
f=open(constellation+"_"+target.replace(constellation, "")+"_"+'%-8.4f'%(t0_ap[i].jd-2400000+ltt0_helio[i].to_value("jd"))+"_"+"DES"+"_MiniMax"+".txt","w")
f.write("#TYPE=BAVMiniMax")
f.write("\n")
f.write("#Delim=|")
f.write("\n")
for line in maxlist:
    f.write(line)
    f.write("\n")
for line in minlist:
    f.write(line)
    f.write("\n")
f.close()

In [ ]:
np.savetxt(constellation+"_"+target.replace(constellation, "")+"_"+'%-8.4f'%(t0_ap[0].jd-2400000+ltt0_helio[0].to_value("jd"))+"_"+"DES"+"_BAVReport"+".txt",lc,fmt="%-8.4f",header="BAV-Report"+"\n"+"Rem=")

In [ ]:
data=np.array([(2442635.374,2448500.0334,2454388.5221,2454388.0832),(0.30864291,0.3086876,0.308626840,0.30862741)])
data=np.transpose(data)

In [ ]:
data=np.array([(2442635.374,2448500.0334,2446342.6620,2454388.0832),(0.30864291,0.3086876,0.30868420,0.30862741)])
data=np.transpose(data)

In [ ]:
elements=pd.DataFrame(data,columns=["epoch","period"],index=["AAVSO","GCVS","BAV","DES"])

In [ ]:
elements

In [ ]:
tmax=(t0_ap[1].to_value("jd")+ltt0_helio[1].to_value("jd"))

In [ ]:
oc=pd.DataFrame(data=None,index=["AAVSO","GCVS","BAV","DES"],columns=["O-C"])
for dataset in elements.itertuples():
    e=int((tmax-dataset.epoch)/dataset.period)
    oc0=tmax-(dataset.epoch+e*dataset.period)
    oc1=tmax-(dataset.epoch+(e+1)*dataset.period)
    if abs(oc0) < abs(oc1):
        oc.loc[dataset.Index]["O-C"]=oc0
    else:
        oc.loc[dataset.Index]["O-C"]=oc1
oc

In [ ]:
ax = plt.subplot(111, frame_on=False) # no visible frame
ax.xaxis.set_visible(False)  # hide the x axis
ax.yaxis.set_visible(False)  # hide the y axis

table(ax, oc)  # where df is your data frame

plt.show()

In [ ]:
pip install astropy